In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


In [55]:
crime_data = pd.read_csv('Crime_Data_from_2020_to_Present.csv')
crime_data.head()
print(crime_data.shape)

In [56]:
conn = sqlite3.connect('crime_data.db')
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS crime_data (
        DR_NO TEXT,
        Date_Rptd TEXT,
        DATE_OCC TEXT,
        TIME_OCC INTEGER,
        AREA INTEGER,
        AREA_NAME TEXT,
        Rpt_Dist_No INTEGER,
        "Part 1-2" INTEGER,  
        Crm_Cd INTEGER,
        Crm_Cd_Desc TEXT,
        Vict_Age INTEGER,
        Vict_Sex TEXT,
        Vict_Descent TEXT,
        Premis_Cd INTEGER,
        Premis_Desc TEXT,
        Weapon_Used_Cd INTEGER,
        Weapon_Desc TEXT,
        Status TEXT,
        Status_Desc TEXT,
        Crm_Cd_1 INTEGER, 
        LOCATION TEXT,  
        LAT REAL,
        LON REAL,
        Year INTEGER,
        Hour INTEGER,
        Weekday INTEGER
    ) 
''')
conn.commit()

In [57]:
# Preprocess data
cleaned_crime = crime_data.sample(n=100000, random_state=42)

# Drop non-essential columns
cleaned_crime = cleaned_crime.drop(['Mocodes', 'Cross Street', 'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4'], axis=1)

# Convert dates to datetime
cleaned_crime['DATE OCC'] = pd.to_datetime(cleaned_crime['DATE OCC'], errors='coerce')
cleaned_crime['Date Rptd'] = pd.to_datetime(cleaned_crime['Date Rptd'], errors='coerce')

# Impute missing categorical data
cleaned_crime['Vict Sex'].fillna('Unknown', inplace=True)
cleaned_crime['Vict Descent'].fillna('Unknown', inplace=True)
cleaned_crime['Weapon Desc'].fillna('Unknown', inplace=True)

# Impute missing numerical data
cleaned_crime['Vict Age'].fillna(cleaned_crime['Vict Age'].median(), inplace=True)
cleaned_crime['Weapon Used Cd'].fillna(0, inplace=True)

# Drop rows with missing critical data
cleaned_crime = cleaned_crime.dropna(subset=['DATE OCC', 'LAT', 'LON'])

# Add additional features if necessary
cleaned_crime['Year'] = cleaned_crime['DATE OCC'].dt.year
cleaned_crime['Hour'] = cleaned_crime['TIME OCC'] // 100
cleaned_crime['Weekday'] = cleaned_crime['DATE OCC'].dt.weekday

# Fill missing values or remove unnecessary columns
cleaned_crime.fillna({'LAT': 0, 'LON': 0}, inplace=True)

# Filter data for valid coordinates
cleaned_crime = cleaned_crime[(cleaned_crime['LAT'] != 0) & (cleaned_crime['LON'] != 0)]

cleaned_crime.info()

In [58]:
cleaned_crime.head()
cleaned_crime.isna().sum()
cleaned_crime.columns


In [59]:
cleaned_crime['DATE OCC'] = cleaned_crime['DATE OCC'].dt.strftime('%Y-%m-%d %H:%M:%S')
cleaned_crime['Date Rptd'] = cleaned_crime['Date Rptd'].dt.strftime('%Y-%m-%d %H:%M:%S')

data_tuples = cleaned_crime.values.tolist()

cursor.executemany('''
    INSERT INTO crime_data (
       DR_NO, Date_Rptd, DATE_OCC, TIME_OCC, AREA, AREA_NAME,
       Rpt_Dist_No, "Part 1-2", Crm_Cd, Crm_Cd_Desc, Vict_Age,
       Vict_Sex, Vict_Descent, Premis_Cd, Premis_Desc,
       Weapon_Used_Cd, Weapon_Desc, Status, Status_Desc, Crm_Cd_1,
       LOCATION, LAT, LON, Year, Hour, Weekday
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
''', data_tuples)
conn.commit()

In [60]:
cursor.execute('PRAGMA table_info(crime_data)')
columns = cursor.fetchall()
print(columns)

In [61]:
cursor.execute('PRAGMA table_info(crime_data)')
columns_info = cursor.fetchall()
print(columns_info)

In [62]:
cursor.execute('SELECT * FROM crime_data')
crimes = cursor.fetchall()

for crime in crimes[:5]:
    print(crime)

cursor.execute('SELECT COUNT(*) FROM crime_data')
counts = cursor.fetchone()[0]
print(counts)

cursor.execute('''
    SELECT Year, COUNT(*) AS crime_count
    FROM crime_data
    GROUP BY Year
    ORDER BY Year
''')
crimes_per_year = cursor.fetchall()

# Print the result
print(crimes_per_year)

In [63]:
cursor.execute('''SELECT Crm_Cd, COUNT(*) AS crime_count
FROM crime_data
GROUP BY Crm_Cd
ORDER BY crime_count DESC
LIMIT 10''')
most_common_crime_codes = cursor.fetchall()
print(most_common_crime_codes)

In [64]:
cursor.execute('''SELECT *
FROM crime_data
WHERE AREA = 1 LIMIT 10''')
area_1 = cursor.fetchall()
print(area_1)

In [65]:
cursor.execute('''SELECT Crm_Cd_Desc, AVG(Vict_Age) AS average_victim_age
FROM crime_data
GROUP BY Crm_Cd_Desc
ORDER BY average_victim_age DESC''')

victim_ages = cursor.fetchall()
print(victim_ages)

In [66]:
cursor.execute('''SELECT LON, COUNT(*) AS crime_count
FROM crime_data
GROUP BY Weekday
ORDER BY crime_count DESC''')

total_crimes_weekdays = cursor.fetchall()
print(total_crimes_weekdays)

In [67]:
cursor.execute('''SELECT AREA_NAME, COUNT(*) AS crime_count
FROM crime_data
GROUP BY AREA_NAME
ORDER BY crime_count DESC''')

total_crimes_in_areas = cursor.fetchall()
print(total_crimes_in_areas)

In [68]:
cursor.execute('''SELECT *
FROM crime_data
WHERE DATE_OCC BETWEEN "2020-01-01" AND "2020-12-31"
''')
crimes_2020 = cursor.fetchall()
print(crimes_2020[:5])

In [69]:
cursor.execute('''SELECT Weapon_Used_Cd, COUNT(*) AS crime_count
FROM crime_data
GROUP BY Weapon_Used_Cd
ORDER BY crime_count DESC
''')
crimes_by_weapon = cursor.fetchall()
print(crimes_by_weapon)

In [70]:
cursor.execute('''SELECT Vict_Descent, COUNT(*) AS crime_count
FROM crime_data
GROUP BY Vict_Descent
ORDER BY crime_count DESC
LIMIT 5''')
most_common_descent_victims = cursor.fetchall()
print(most_common_descent_victims)

In [71]:
# Convert 'DATE OCC' to datetime format (if it's not already)
cleaned_crime['DATE OCC'] = pd.to_datetime(cleaned_crime['DATE OCC'], errors='coerce')

# Now, extract the year
cleaned_crime['Year'] = cleaned_crime['DATE OCC'].dt.year
yearly_crime = cleaned_crime.groupby('Year').size()


plt.figure(figsize=(10,6))
sns.lineplot(x=yearly_crime.index, y=yearly_crime.values)
plt.title("Crime Trend over the Years")
plt.xlabel("Year")
plt.ylabel("Number of Crimes")
plt.show()

In [72]:
crime_by_weekday = cleaned_crime.groupby('Weekday').size()

plt.figure(figsize=(10,6))
sns.barplot(x=crime_by_weekday.index, y=crime_by_weekday.values)
plt.title("Crime Frequency by Weekday")
plt.xlabel("Weekday (0=Monday, 6=Sunday)")
plt.ylabel("Number of Crimes")
plt.show()

In [73]:
# Plot crime count by hour of day
crime_by_hour = cleaned_crime.groupby('Hour').size()

plt.figure(figsize=(10,6))
sns.lineplot(x=crime_by_hour.index, y=crime_by_hour.values)
plt.title("Crime Frequency by Hour of Day")
plt.xlabel("Hour of Day")
plt.ylabel("Number of Crimes")
plt.show()

In [74]:
stolen_vehicle_crimes = cleaned_crime[cleaned_crime['Crm Cd'] == 510]

# Plot by weekday, hour, or month
stolen_vehicle_by_weekday = stolen_vehicle_crimes.groupby('Weekday').size()

plt.figure(figsize=(10,6))
sns.barplot(x=stolen_vehicle_by_weekday.index, y=stolen_vehicle_by_weekday.values)
plt.title("Stolen Vehicle Crimes by Weekday")
plt.xlabel("Weekday")
plt.ylabel("Number of Crimes")
plt.show()

In [75]:
query = """
SELECT Vict_Age, Crm_Cd
FROM crime_data
WHERE Vict_Age IS NOT NULL AND Crm_Cd IS NOT NULL
"""

# Load crime data
df = pd.read_sql(query, conn)

# Close database connection
conn.close()

# Aggregate crime counts by age group and crime code
# We categorize ages into groups for better visualization (optional)
df['Age_Group'] = pd.cut(df['Vict_Age'], bins=[0, 18, 35, 50, 65, 100], 
                          labels=["0-18", "19-35", "36-50", "51-65", "66+"])

# Create the box plot
plt.figure(figsize=(12, 7))
sns.boxplot(x="Age_Group", y="Crm_Cd", data=df, palette="coolwarm")

# Add title and axis labels
plt.title("Crime Categories Across Victim Age Groups")
plt.xlabel("Victim Age Group")
plt.ylabel("Crime Code")
plt.xticks(rotation=45)
plt.tight_layout()

# Display the plot
plt.show()

In [76]:
# Pivot data to create a heatmap
crime_heatmap = cleaned_crime.pivot_table(index='Weekday', columns='Hour', aggfunc='size')

plt.figure(figsize=(12,6))
sns.heatmap(crime_heatmap, cmap="YlGnBu", annot=True, fmt="d", cbar_kws={'label': 'Crime Count'})
plt.title("Crime Frequency by Hour and Weekday")
plt.xlabel("Hour of Day")
plt.ylabel("Weekday")
plt.show()

In [77]:
top_crimes = cleaned_crime['Crm Cd Desc'].value_counts().head(10)

plt.figure(figsize=(12, 6))
sns.countplot(y='Crm Cd Desc', data=cleaned_crime, order=top_crimes.index)
plt.title("Top 10 Crime Categories")
plt.xlabel("Count")
plt.ylabel("Crime Category")
plt.show()

In [78]:
# Scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(x='LON', y='LAT', data=cleaned_crime, alpha=0.5)
plt.title("Crime Locations (Scatter Plot)")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

In [79]:
# crime density using kde plot
plt.figure(figsize=(10, 6))
sns.kdeplot(
    x='LON',
    y='LAT',
    data=cleaned_crime,
    fill=True,
    cmap='Reds',
    alpha=0.6
)
plt.title("Crime Density in Los Angeles")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

In [80]:
from sklearn.preprocessing import LabelEncoder

# Label encode 'Vict Sex' (binary categorical variable)
label_encoder = LabelEncoder()
cleaned_crime['Vict Sex'] = label_encoder.fit_transform(cleaned_crime['Vict Sex'])

# One-hot encode 'Vict Descent' (if you want to handle it as categorical)
cleaned_crime = pd.get_dummies(cleaned_crime, columns=['Vict Descent'], drop_first=True)

# Handle other categorical variables: 'Premis Cd', 'Weapon Used Cd', 'Status', 'AREA', 'Rpt Dist No'
cleaned_crime = pd.get_dummies(cleaned_crime, columns=['Premis Cd', 'Weapon Used Cd', 'Status', 'AREA'], drop_first=True)

# Now 'Vict Sex' is encoded and 'Vict Descent' has been one-hot encoded
# You can now safely select the features and target
X = cleaned_crime[['Hour', 'Weekday', 'LAT', 'LON', 'Vict Age', 'Vict Sex', 'Year', 'Rpt Dist No'] + 
                  [col for col in cleaned_crime.columns if 'Vict Descent' in col]]
y = (cleaned_crime['Crm Cd'] == 510).astype(int)  # Example: Predicting stolen vehicle crimes

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [81]:
from sklearn.cluster import KMeans

coords = cleaned_crime[['LAT', 'LON']]

kmeans = KMeans(n_clusters=5, random_state=42)
coords['Cluster'] = kmeans.fit_predict(coords)

plt.figure(figsize=(10,8))
for cluster in range(5):
    cluster_points = coords[coords['Cluster'] == cluster]
    plt.scatter(cluster_points['LAT'], cluster_points['LON'], label=f'Cluster {cluster}')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:,1], s=200, c='red', marker='X',label='Centroids')
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.title('Crime Hotspot Clustering')
plt.legend()
plt.show()

In [82]:
from sklearn.ensemble import IsolationForest

features = cleaned_crime[['Hour', 'Weekday', 'LAT', 'LON', 'Year']]

isolation_forest = IsolationForest(contamination=0.01, random_state=42)
features['Anomaly'] = isolation_forest.fit_predict(features)

anomalies = features[features['Anomaly'] == -1]
plt.figure(figsize=(10,8))
plt.scatter(features['LAT'], features['LON'], c='blue', label='Normal')
plt.scatter(anomalies['LAT'], anomalies['LON'], c='red', label='Anomalies')
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.title('Crime Anomaly Detection')
plt.legend()
plt.show()

In [ ]:
# Attempted Apache Airflow that we researched

from airflow import DAG
from airflow.operators.python import PythonOperator
from sklearn.linear_model import LogisticRegression
from datetime import datetime, timedelta
import joblib
from functools import partial


def extract_data(file):
    # Load crime data from a CSV file
    df = pd.read_csv(file)
    df.to_csv('/tmp/crime_data_extracted.csv', index=False)

def clean_data():
    # Read extracted data
    df = pd.read_csv('/tmp/crime_data_extracted.csv')
    df['DATE_OCC'] = pd.to_datetime(df['DATE_OCC'])
    df['Year'] = df['DATE_OCC'].dt.year
    df.to_csv('/tmp/crime_data_cleaned.csv', index=False)

def load_data():
    # Load cleaned data into SQLite database
    conn = sqlite3.connect('/tmp/crime_data.db')
    df = pd.read_csv('/tmp/crime_data_cleaned.csv')
    df.to_sql('crime_data', conn, if_exists='replace', index=False)

def train_model():
    # Train a Logistic Regression model on the crime dataset
    conn = sqlite3.connect('/tmp/crime_data.db')
    query = "SELECT Hour, Weekday, LAT, LON, Vict_Age, Crm_Cd FROM crime_data"
    df = pd.read_sql(query, conn)
    conn.close()

    # Feature selection and target preparation
    X = df[['Hour', 'Weekday', 'LAT', 'LON', 'Vict_Age']].dropna()
    y = (df['Crm_Cd'] == 510).astype(int)  # Example: Predict if the crime code is '510'

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Logistic Regression model
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)

    # Save the model to a file
    joblib.dump(model, '/tmp/crime_model.pkl')

    # Print model performance
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred)
    print("Classification Report:\n", report)

default_args = {
    'owner': 'team',
    'depends_on_past': False,
    'start_date': datetime(2024, 12, 1),
    'retries': 1,
    'retry_delay': timedelta(minutes=10),
}
dag = DAG('crime_analysis_pipeline', default_args=default_args, schedule_interval=None)

# Use partial to pass the file parameter to extract_data
extract_task = PythonOperator(
    task_id='extract_data',
    python_callable=partial(extract_data, file='Crime_Data_from_2020_to_Present.csv'),
    dag=dag
)
clean_task = PythonOperator(task_id='clean_data', python_callable=clean_data, dag=dag)
load_task = PythonOperator(task_id='load_data', python_callable=load_data, dag=dag)
train_task = PythonOperator(task_id='train_model', python_callable=train_model, dag=dag)

# Set dependencies
extract_task >> clean_task >> load_task >> train_task